In [2]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode


def read_vocabulary(file_path):
    with open(file_path, 'r') as infile:
        return set(infile.read().splitlines())

stop_words = read_vocabulary("stop_words.txt")


for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        if word in stop_words:
            continue
        word = word.lower()
        print "%s\t%s\t%d" % (sorted(word), word, 1)


Overwriting mapper.py


In [5]:
%%writefile reducer.py

import sys
from collections import Counter

current_key = None
word_sum = 0
words_batch = set()


for line in sys.stdin:
    try:
        key, word, count = line.strip().split('\t', 2)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key and len(words_batch) > 1:
            print "%d\t%d\t%s" % (word_sum, len(words_batch), ','.join(sorted(words_batch)))
        words_batch = set()
        word_sum = 0
        current_key = key
    words_batch.add(word)
    word_sum += count

if current_key and len(words_batch) > 1:
    print "%d\t%d\t%s" % (word_sum, len(words_batch), ','.join(sorted(words_batch)))


Overwriting reducer.py


In [7]:
%%bash

NUM_REDUCERS=8

IN_DIR="/data/wiki/en_articles_part"
OUT_DIR="group_count_result"  # $(date +"%s%6N")

hdfs dfs -rm -r -f -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming group word count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt#stop_words.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input ${IN_DIR} \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -text ${OUT_DIR}/part-* | grep english | sort -k1rn | head -1

7825	5	english,helsing,hesling,shengli,shingle


17/10/17 12:57:08 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/10/17 12:57:08 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/10/17 12:57:09 INFO mapred.FileInputFormat: Total input files to process : 1
17/10/17 12:57:09 INFO mapreduce.JobSubmitter: number of splits:2
17/10/17 12:57:09 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
17/10/17 12:57:09 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1508231375397_0034
17/10/17 12:57:10 INFO impl.YarnClientImpl: Submitted application application_1508231375397_0034
17/10/17 12:57:10 INFO mapreduce.Job: The url to track the job: http://843f47db8d38:8088/proxy/application_1508231375397_0034/
17/10/17 12:57:10 INFO mapreduce.Job: Running job: job_1508231375397_0034
17/10/17 12:57:17 INFO mapreduce.Job: Job job_1508231375397_0034 running in uber mode : false
17/10/17 12:57:17 INFO mapreduce.Job:  map 0% reduce 0%
17/10/17 12:57:35 INFO m